In [53]:
import os, nltk
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [35]:
dataset = pd.read_csv("../resources/datasets/TwitterAirlines/TweetsProcessed.csv", index_col=0)
dataset

,text,class
570306133677760513,said,1
570301130888122368,plus ve added commercial experience tacky,2
570301083672813571,nt today must mean need take another trip,1
570301031407624196,s really aggressive blast obnoxious entertainm...,0
570300817074462722,s really big bad thing,0
...,...,...
569587686496825344,thank got different flight chicago,2
569587371693355008,leaving minute late flight warning communicati...,0
569587242672398336,please bring american airline,1
569587188687634433,money change flight nt answer phone suggestion...,0


In [37]:
sentences = []
for i, sentence in enumerate(list(dataset["text"])) :
    # print(i, sentence, type(sentence))
    sentences.append(nltk.word_tokenize(sentence))

In [45]:
model = Word2Vec(sentences=sentences, vector_size=200, window=5, min_count=1, workers=4)
model.train(sentences, epochs=10, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True)

(1140670, 1295570)

In [54]:
words = model.wv.index_to_key
vectors = []
for word in words :
    vectors.append(model.wv[word])
vectors = np.array(vectors)

In [56]:
embeddings = pd.DataFrame(vectors, index=words, columns=range(1, 201))

In [57]:
embeddings

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
flight,0.731328,-0.562065,0.009106,0.570078,-0.092396,-0.845431,0.517720,0.860881,-0.235601,0.630860,...,0.287283,0.170004,0.841820,-0.682707,0.010573,-0.473068,0.633810,-0.240196,-0.623491,0.425027
nt,0.479356,-0.092841,-0.864545,0.344899,0.442046,-0.143405,0.129590,0.719486,-0.978185,1.341606,...,-0.684079,0.381241,0.148041,-0.413139,1.206284,-0.497338,1.015853,-0.538233,-0.357926,-0.720001
s,0.234438,-0.285000,-0.375144,0.376870,0.482242,-0.278014,0.225591,0.537746,-0.209728,0.123051,...,0.150215,0.071122,-0.195098,-0.020906,0.260056,0.317123,-0.038797,-0.120501,0.001339,-0.426071
get,0.267672,-0.189836,-0.055125,0.181075,-0.012418,-0.471935,0.439834,0.722727,0.491443,0.837400,...,0.336407,0.222624,-0.224759,0.156864,0.567207,-0.189672,1.588512,-0.574646,-0.506462,-0.606499
hour,0.018333,0.051522,-0.462388,0.982241,-0.018369,-0.930689,0.711520,1.134589,-0.191998,1.565754,...,0.086593,0.323186,0.597066,0.465976,0.168107,-0.272919,0.405325,-0.114727,-0.009559,0.051134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nelson,0.026116,-0.013890,0.000166,0.055798,0.033110,-0.032027,-0.005012,0.050368,-0.043504,0.021898,...,0.033102,0.003263,-0.014456,-0.013334,0.064210,0.036183,0.008045,-0.066740,-0.042743,-0.036357
impersonator,0.025456,-0.018953,-0.008409,0.047668,0.034288,-0.028115,-0.002097,0.052900,-0.043150,0.022670,...,0.034285,-0.003142,-0.017524,-0.013698,0.061203,0.021549,0.012197,-0.060622,-0.038550,-0.037811
sang,0.018957,-0.015292,-0.006751,0.043520,0.025349,-0.027196,-0.003878,0.044442,-0.045300,0.021592,...,0.026345,0.001638,-0.006219,-0.009284,0.046308,0.034811,0.012416,-0.047661,-0.027534,-0.030086
skymall,0.008385,-0.017745,-0.012027,0.024030,0.019898,-0.023706,0.001752,0.031599,-0.025159,0.008477,...,0.017769,-0.006588,-0.012484,-0.004170,0.028007,0.015200,-0.010117,-0.030066,-0.025667,-0.016228
